In [21]:
try:
    if is_init:
        pass
except:
    %pwd
    %cd ..
    %pwd
    is_init = True

In [22]:
from pathlib import Path
from src.utils import FileUtils
from config import Config
from utils import rmse
import torch
import numpy as np
import pandas as pd
import altair as alt
import random
import itertools
data_folder = '../local_data'
work_folder = '../local_work'
configs_folder = '../configs'

In [23]:
def denormalize_decode_result(y_hat, max_speed, max_spacing, min_speed, min_spacing):
    """input: 1,14 tensor
        output: 1,3 tensor with wattage no longer one hot encoded"""
    watts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

    watt_arg = torch.argmax(y_hat[0][2:])
    y_final = torch.empty(3, dtype=torch.float32)
    y_final[0] = max_speed * y_hat[0][0] # TODO call the scale
    y_final[1] = max_spacing * y_hat[0][1] # TODO call the scale
    y_final[2]= watts[watt_arg]
    return y_final

# Inverse

In [24]:
filepath = Path(f"{work_folder}/saved_best/I-0.9-res-ann-stainless.ckpt") #CHANGEME
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/I-0.9-res-ann-stainless.ckpt


In [25]:
def get_test_data(path):
    stain_data = torch.load(
                    Path(f"{data_folder}/{path}"))#["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
    wave_data = stain_data["interpolated_wavelength"]
    emiss_data = stain_data["interpolated_emissivity"]#.keys()#["wavelenght"]
    #print(wave_test.max(1), wave_test.min(1)) #2.5002, 11.9479
    #print(emiss_test.max(1)[0])#.max(1), emiss_test.min(1)) #0, 1
    wave_test = wave_data[round(len(wave_data) * .95):]  
    emiss_test = emiss_data[round(len(wave_data) * .95):]
    return wave_test, emiss_test
#print(max_wave, min_wave)

In [26]:
# pass test data into inverse model, get predicted parameters
from src.models import InverseModel
i_model = InverseModel.load_from_checkpoint(filepath, strict=False)
i_model.eval()

wave_test, emiss_test = get_test_data('stainless-steel-revised-shuffled.pt')
test_y_hats = []#torch.empty(size = len(emiss_test))
for emiss in emiss_test:
    with torch.no_grad():
        test_y_hats.append(i_model(emiss.reshape(1, 800)))

/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


TypeError: 'NoneType' object is not callable

In [ ]:
test_y_hats[0]

tensor([[0.1704, 0.3627, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

# Direct Model

In [28]:
from src.models import DirectModel
direct_filepath = Path(""
    f"{work_folder}/saved_best/best_D-ann-inconel-2022-07-05_14-39.ckpt") #CHANGEME
print(direct_filepath)
if not Path.is_file(direct_filepath):
    raise Exception(f'Model file does not exist at {direct_filepath}!')

AttributeError: module 'wandb.sdk' has no attribute 'lib'

In [ ]:
d_model = DirectModel.load_from_checkpoint(direct_filepath)
d_model.eval()
emiss_y_hats = []
for x in test_y_hats:
    with torch.no_grad():
        emiss_y_hats.append(d_model(x.reshape(1,14)))

In [ ]:
wave_test[1].tolist()

[2.5001838207244873,
 2.5120081901550293,
 2.5238325595855713,
 2.5356569290161133,
 2.5474812984466553,
 2.5593059062957764,
 2.5711302757263184,
 2.5829546451568604,
 2.5947790145874023,
 2.6066033840179443,
 2.6184277534484863,
 2.6302521228790283,
 2.6420767307281494,
 2.6539011001586914,
 2.6657254695892334,
 2.6775498390197754,
 2.6893742084503174,
 2.7011985778808594,
 2.7130229473114014,
 2.7248475551605225,
 2.7366719245910645,
 2.7484962940216064,
 2.7603206634521484,
 2.7721450328826904,
 2.7839694023132324,
 2.7957937717437744,
 2.8076183795928955,
 2.8194427490234375,
 2.8312671184539795,
 2.8430914878845215,
 2.8549158573150635,
 2.8667402267456055,
 2.8785645961761475,
 2.8903892040252686,
 2.9022135734558105,
 2.9140379428863525,
 2.9258623123168945,
 2.9376866817474365,
 2.9495110511779785,
 2.9613354206085205,
 2.9731600284576416,
 2.9849843978881836,
 2.9968087673187256,
 3.0086331367492676,
 3.0204575061798096,
 3.0322818756103516,
 3.0441062450408936,
 3.0559308528

In [ ]:
#calculate rmse
pred_rmse_vals = []
columns = ["wavelength", "emissivity", "prediction", "example_number", "laser parameters"]
viz_coords = pd.DataFrame(columns = columns)
for i in range(9):#len(emiss_test)):
    pred_rmse_vals.append(rmse(emiss_y_hats[i], emiss_test[i]).item())
    for j in range(len(wave_test[i].tolist())):
        viz_coords.append([wave_test[i].tolist()[j], emiss_test[i].tolist()[j], "True", i])
        viz_coords.append([wave_test[i].tolist()[j], emiss_y_hats[i].flatten().tolist()[j], "Prediction", i])
stdev = np.std(pred_rmse_vals)
print(f'Direct Filepath: {direct_filepath}')
print(f'Inverse Filepath: {filepath}')
print(f'SD of RMSEs on test data: {stdev}')


NameError: name 'pd' is not defined

In [ ]:
alt.Chart(viz_coords).mark_line().encode(
    x='wavelength',
    y='emissivity'
    legend=
).facet(facet = 'example_number:N')

In [ ]:
#visualize predictions vs original
